# `torch.autograd` 自动微分

在训练神经网络时，最常用的算法是反向传播(back propagation)。为了计算这些梯度，PyTorch 内置了一个名为 `torch.autograd` 的微分引擎。 考虑最简单的单层神经网络，输入 `x`、参数 `w` 和 `b` 以及一些损失函数。 在 PyTorch 中，它可以按以下方式定义：

In [1]:
import torch

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## 张量、函数和计算图

上面代码定义了以下计算图：

<div align="center">
    <img src="./comp-graph.png" alt="计算图" />
</div>

在这个网络中，`w` 和 `b` 是我们需要优化的参数。因此，我们需要计算损失函数相对于这些变量的梯度。为此，我们设置了这些张量的 `requires_grad` 属性。

> 您可以在创建张量时设置 `requires_grad` 的值，也可以稍后使用 `x.requires_grad_(True)` 方法来设置。

我们应用张量来构建计算图的函数实际上是一个函数类对象。该对象知道如何在前向计算函数，以及如何在反向传播步骤中计算其导数。反向传播函数的引用存储在张量的 `grad_fn` 属性中。有关函数的更多信息，请参阅[文档](https://pytorch.org/docs/stable/autograd.html#function)。

In [2]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7f2a736b7430>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f2a736b7310>


## 计算梯度

要优化神经网络中的参数权重，我们需要计算损失函数关于参数的导数，即我们需要$\frac{\partial{loss}}{\partial x}$和$\frac{\partial{loss}}{\partial b}$在一些固定的 `x` 和 `y` 值下。要计算这些导数，我们需要调用 `loss.backward()`，然后从 `w.grad` 和 `b.grad` 中获取数值：

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1882, 0.2939, 0.0246],
        [0.1882, 0.2939, 0.0246],
        [0.1882, 0.2939, 0.0246],
        [0.1882, 0.2939, 0.0246],
        [0.1882, 0.2939, 0.0246]])
tensor([0.1882, 0.2939, 0.0246])


[ROCT-Thunk-Interface/src/hymgr.c:309, WARNING]: Version mismatch


> + 我们只能获取计算图中叶子节点的梯度属性，这些节点的 `requires_grad` 属性设置为 `True`。对于图中的所有其他节点，梯度将不可用。
> + 出于性能考虑，我们只能在给定的计算图上使用反向调用一次梯度计算。如果我们需要在同一图形上执行多次 `backward` 调用，则需要向 `backward` 调用传递 `retain_graph=True` 参数。

## 关闭梯度追踪

默认情况下，所有 `requires_grad=True` 的张量都会跟踪其计算历史并支持梯度计算。不过，在某些情况下，我们并不需要这样做，例如，当我们已经训练好模型，只想将其应用于某些输入数据时，也就是说，我们只想通过网络进行前向计算。我们可以用 `torch.no_grad()` 代码块**包围**我们的计算代码，从而停止跟踪计算：

In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


另一种实现相同效果的方法是在张量上使用 `detach()` 方法：

In [5]:
z = torch.matmul(x, w)+b
print(z.requires_grad)
z_det = z.detach()
print(z_det.requires_grad)

True
False


您可能会出于以下原因**禁用梯度跟踪**：
+ 将神经网络中的某些参数标记为冻结参数(frozen parameters)。
+ 在只进行前向传递时加快计算速度，因为对不跟踪梯度的张量进行计算会更有效率。

## 计算图

从概念上讲，autograd 将数据（张量）和所有已执行的操作（以及产生的新张量）记录在由 [Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function) 对象组成的有向无环图（DAG）中。 在这个 DAG 中，叶是输入张量，根是输出张量。通过从根追踪到叶的图，可以使用**链式法则**自动计算梯度。

在前向传递中，autograd 同时做两件事：

+ 运行所请求的操作以计算出结果张量；
+ 并在 DAG 中维护操作的梯度函数；

当在 DAG 根上调用 `.backward()` 时，反向传递开始：

+ 计算每个 `.grad_fn` 的梯度；
+ 利用链式规则将梯度累加到相应张量的 `.grad` 属性中；
+ 并一直传播到叶张量；

> 在 PyTorch 中，DAG 是动态的。需要注意的是，图形是从头开始创建的；每次调用 `.backward()` 之后，`autograd` 都会开始填充一个新的图形。这正是你在模型中使用控制流语句的原因；如果需要，你可以在每次迭代时改变图形的形状、大小和操作。

## 张量梯度和 Jacobian 乘积

在很多情况下，我们有一个标量损失函数，需要计算与某些参数相关的梯度。然而，在某些情况下，输出函数是一个任意的张量。在这种情况下，PyTorch 允许你计算所谓的雅各布乘积，而不是实际梯度。

对于向量函数$\vec{y}=f(\vec{x})$，其中 $\vec{x}=<x_1,...,x_n>$ 和 $\vec{y}=<y_1,...,y_m>$，那么 $\vec{y}$ 相对于 $\vec{x}$ 的梯度通过雅各布矩阵：

$$
\mathbf{J} = \begin{pmatrix}
\frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} & \cdots & \frac{\partial y_1}{\partial x_n} \\
\frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} & \cdots & \frac{\partial y_2}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial y_m}{\partial x_1} & \frac{\partial y_m}{\partial x_2} & \cdots & \frac{\partial f_m}{\partial x_n}
\end{pmatrix}
$$


PyTorch 不计算雅各布矩阵本身，而是允许你为给定的输入向量$v=(v_1,...,v_m)$计算雅各布积乘积$v^T\cdot\mathbf{J}$。 这是通过以$v$为参数调用 `backward` 来实现的。$v$的大小应与我们要计算乘积的原始张量的大小相同：

In [6]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp + 1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients call\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


请注意，当我们使用相同的参数第二次调用后向传播时，梯度值是不同的。这是因为在进行反向传播时，PyTorch 会**累积梯度**，也就是说，计算出的梯度值会添加到计算图中所有叶节点的 `grad` 属性中。如果你想计算适当的梯度，就需要先将 `grad` 属性清零。在实际训练中，优化器可以帮助我们做到这一点。

> 在此之前，我们调用的是不带参数的 `backward()` 函数。这本质上等同于调用 `backward(torch.tensor(1.0))`，这是在标量值函数（如神经网络训练过程中的损失）情况下计算梯度的有用方法。

## Further Reading

+ [Autograd Mechanics](https://pytorch.org/docs/stable/notes/autograd.html)